<a href="https://colab.research.google.com/github/rendzina/BigDataAndVisualisation/blob/main/Colab/MongoDB_Aggregation_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MongoDB Aggregations

*MKU: Big Data and Visualisation*

Here's how to do aggregations in MongoDB in Python using Colab

In [ ]:
# First, you need to install the pymongo library, which isn't there by default
!pip install pymongo

In [ ]:
# Now we can install the library for use
import pymongo
from pymongo import MongoClient

In [ ]:
# Next we set up the connection to the database and set the credentials
client = pymongo.MongoClient("mongodb+srv://<USER>:<PASSWORD>@<CLUSTERNAME>?retryWrites=true&w=majority")
db = client.environment
collection = db.noise_mapping

In [ ]:
# Then we can bujild the aggregation itself, connecting to the mongodb host. The results are placed in the 'cursor' object.
cursor = collection.aggregate([
  {
    "$match": {"Location/Agglomeration":"Milton Keynes Urban Area"}
  },
  {
    "$project" :  {
      "Location/Agglomeration": 1,
      "Road_Pop_Lden>=70dB": 1,
      "Railways_Pop_Lden>=70dB": 1,
      "Affects_More_People?":
        {
          "$switch":
            {
              "branches":[
                {
                    "case": {"$gt" : ["$Road_Pop_Lden>=70dB", "$Railways_Pop_Lden>=70dB"]}, "then": "Road"
                },
                {
                    "case": {"$gt" : ["$Railways_Pop_Lden>=70dB", "$Road_Pop_Lden>=70dB"]}, "then": "Rail"
                },
               ],
                "default": "No Difference"
            }
        }
    }
}])

In [ ]:
# Finally we can inspect the cursor contents with the list command
list(cursor)

[{'_id': ObjectId('663a05011147d3732dd23e1e'),
  'Location/Agglomeration': 'Milton Keynes Urban Area',
  'Road_Pop_Lden>=70dB': 200,
  'Railways_Pop_Lden>=70dB': 600,
  'Affects_More_People?': 'Rail'}]